In [1]:
import os, sys; sys.path.append(os.path.dirname(os.getcwd())); ROOT_DIR = '/home/h4zzkr/lab/delOS/'
dataset_path = os.path.join(ROOT_DIR, 'data/nlu_data/custom')

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path

from transformers import BertTokenizer
from transformers import TFBertModel

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
df_train = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
df_valid = pd.read_csv(os.path.join(dataset_path, 'valid.csv'))

In [4]:
intent_names = Path(os.path.join(dataset_path, "vocab.intent")).read_text().split()
intent_map = dict((label, idx) for idx, label in enumerate(intent_names))
intent_train = df_train["intent_label"].map(intent_map).values
intent_valid = df_valid["intent_label"].map(intent_map).values

In [5]:
id2intent = {intent_map[k] : k for k in intent_map.keys()}

In [6]:
from sentence_transformers import SentenceTransformer
featurizer = SentenceTransformer('bert-base-nli-mean-tokens')

In [7]:
encoded_train = np.array(featurizer.encode(df_train['words']))
encoded_valid = np.array(featurizer.encode(df_valid['words']))

In [8]:
def accuracy(model, y_true, valid_data):
    y_pred = np.argmax(model.predict_proba(valid_data), axis=1)
    return accuracy_score(y_true, y_pred)

In [38]:
lgmodel = LogisticRegression(
    penalty='l1',
    C=1,
    tol=0.00001,
    solver='liblinear',
    max_iter=2,
)
lgmodel.fit(encoded_train, intent_train)
accuracy(lgmodel, intent_valid, encoded_valid)

/home/h4zzkr/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9090909090909091

In [57]:
from xgboost import XGBClassifier
xgbmodel = XGBClassifier(
    max_depth=2,
    gamma=0.5,
)
xgbmodel.fit(encoded_train, intent_train)
accuracy(xgbmodel, intent_valid, encoded_valid)

0.9090909090909091

In [9]:
import tensorflow as tf
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.activations import softmax

In [10]:
class IntentClassificationModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, dropout_prob=0.15):
        super().__init__(name="")
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels)

    def call(self, inputs, **kwargs):
        inputs = self.dropout(inputs)
        out = self.intent_classifier(inputs)
        out = softmax(out)
        return out


densemodel = IntentClassificationModel(intent_num_labels=len(intent_map))
optim = Adam(learning_rate=0.001, epsilon=1e-09)
loss_f = SparseCategoricalCrossentropy(from_logits=False)
metrics_f = [SparseCategoricalAccuracy('accuracy')]

densemodel.compile(optimizer=optim,
                     loss=loss_f,
                     metrics=metrics_f
                    )

In [11]:
history = densemodel.fit(encoded_train, intent_train, epochs=9, batch_size=32,
                           validation_data=(encoded_valid, intent_valid))

Epoch 1/9
2/2 [==============================] - 0s 83ms/step - loss: 0.8220 - accuracy: 0.4318 - val_loss: 0.7393 - val_accuracy: 0.5000
Epoch 2/9
2/2 [==============================] - 0s 16ms/step - loss: 0.6759 - accuracy: 0.6591 - val_loss: 0.5448 - val_accuracy: 0.7500
Epoch 3/9
2/2 [==============================] - 0s 19ms/step - loss: 0.5647 - accuracy: 0.7500 - val_loss: 0.4013 - val_accuracy: 1.0000
Epoch 4/9
2/2 [==============================] - 0s 19ms/step - loss: 0.4295 - accuracy: 0.8409 - val_loss: 0.3030 - val_accuracy: 1.0000
Epoch 5/9
2/2 [==============================] - 0s 19ms/step - loss: 0.3990 - accuracy: 0.8409 - val_loss: 0.2336 - val_accuracy: 1.0000
Epoch 6/9
2/2 [==============================] - 0s 19ms/step - loss: 0.3636 - accuracy: 0.8409 - val_loss: 0.1860 - val_accuracy: 1.0000
Epoch 7/9
2/2 [==============================] - 0s 18ms/step - loss: 0.3071 - accuracy: 0.8636 - val_loss: 0.1546 - val_accuracy: 1.0000
Epoch 8/9
2/2 [===================

In [15]:
def test_output(inp, encoder, model, id2intent):
    inp = np.array(encoder.encode(inp))
    out = np.argmax(np.array(model(inp)))
    return id2intent[out]
    
inp = 'so dark, please turn on the light'
test_output(inp, featurizer, densemodel, id2intent)

'turnLightOn'

In [13]:
def test_acc_output(inp, encoder, model, valid, id2intent):
    clas = test_output(inp, encoder, model, id2intent)
    return clas, valid == clas

In [14]:
for (i, c) in zip(df_valid['words'], df_valid['intent_label']):
    print(i, test_acc_output(i, featurizer, densemodel, c, id2intent))

turn on lights for the little kitchen ('turnLightOn', True)
enable electric lights inside the office ('turnLightOn', True)
cut the lights down in the house ('turnLightOff', True)
turn off the bathroom lights ('turnLightOff', True)
light on ('turnLightOn', True)
turn off the lights next door ('turnLightOff', True)
turn off the light ('turnLightOff', True)
disable the house lights in across the garage ('turnLightOff', True)
disable the illumination in the kitchen please ('turnLightOff', True)
can you turn off the light here ('turnLightOff', True)
turn on light upstairs ('turnLightOn', True)
enable light in the bedroom ('turnLightOn', True)
